In [6]:
import csv
import gzip
import re

In [7]:
from collections import defaultdict

In [8]:
gene_tran = defaultdict(set)
gene_info = dict()

In [9]:
with gzip.open("../data/ref/pan_troglodytes/annot/Pan_troglodytes.Pan_tro_3.0.112.gtf.gz", 'rt') as f:
    for row in f:
        if row.startswith("#"):
            continue

        row = row.strip("\n").split("\t")
        chrom, _, feat, *pos, _, strand, _, meta_data = row

        if feat == "transcript":
            gene = re.search(rf'(?<=gene_id ")\w+', meta_data).group(0)
            tran = re.search(rf'(?<=transcript_id ")\w+', meta_data).group(0)

            beg, end = map(int, pos)

            gene_tran[gene].add(tran)
            gene_info[gene] = beg, end

In [10]:
with open("../data/ref/pan_troglodytes/annot/out/genes.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["gene", "beg", "end", "len", "tran_no"])

    for gene, tran_set in gene_tran.items():
        tran_no = len(tran_set)
        beg, end = gene_info[gene]

        writer.writerow([
            gene,
            beg,
            end,
            end - beg - 1,
            tran_no
        ])